1. Отобразите все записи из таблицы company по компаниям, которые закрылись.

```sql
SELECT *
FROM company
WHERE status = 'closed';
```

2. Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы `company`. Отсортируйте таблицу по убыванию значений в поле `funding_total`.

Чем занимается компания, можно понять по значению в поле `category_code`. Чтобы объединить условия, используйте логический оператор.

```sql
SELECT funding_total
FROM company
WHERE country_code LIKE 'USA' AND  category_code LIKE '%news%'
ORDER BY funding_total DESC;
```

3. Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

Всю информацию нужно взять из таблицы `acquisition`. Способ оплаты сделки указан в поле term_code.

```sql
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code LIKE 'cash'
AND EXTRACT(YEAR FROM CAST (acquired_at AS DATE)) IN (2011, 2012, 2013);
```

4. Отобразите имя, фамилию и названия аккаунтов людей в поле `network_username`, у которых названия аккаунтов начинаются на `'Silver'`.

Получите срез по шаблону.

```sql
SELECT p.first_name,
p.last_name,
p.network_username
FROM people AS p
WHERE p.network_username LIKE 'Silver%';
```

5. Выведите на экран всю информацию о людях, у которых названия аккаунтов в поле network_username содержат подстроку 'money', а фамилия начинается на 'K'.

```sql
SELECT *
FROM people
WHERE network_username LIKE '%money%'
AND last_name LIKE 'K%';
```

6. Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

Используйте агрегирующую функцию и группировку. При сортировке тоже можно обращаться к результату агрегирующей функции.

```sql
SELECT
country_code,
SUM(funding_total) AS total_funding
FROM company
GROUP BY country_code
ORDER BY total_funding DESC;
```

7. Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

```sql
SELECT
fr.funded_at,
MIN(fr.raised_amount) AS min_investment,
MAX(fr.raised_amount) AS max_investment
FROM
funding_round AS fr

GROUP BY
fr.funded_at
HAVING
MIN(fr.raised_amount) <> 0 AND
MIN(fr.raised_amount) <> MAX(fr.raised_amount);
```

8. Создайте поле с категориями:

- Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию `high_activity`.
- Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию `middle_activity`.
- Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию `low_activity`.

Отобразите все поля таблицы `fund` и новое поле с категориями.

Подсказка

Создать новое поле с категориями можно оператором `CASE`.

```sql
SELECT
*,
CASE
WHEN invested_companies >= 100 THEN 'high_activity'
WHEN invested_companies >= 20 THEN 'middle_activity'
ELSE 'low_activity'
END AS activity_category
FROM
fund;
```

9. Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

```sql
SELECT
activity,
ROUND(AVG(investment_rounds)) AS avg_investment_rounds
FROM (
SELECT
*,
CASE
WHEN invested_companies >= 100 THEN 'high_activity'
WHEN invested_companies >= 20 THEN 'middle_activity'
ELSE 'low_activity'
END AS activity
FROM fund
) AS categorized_funds
GROUP BY
activity
ORDER BY
avg_investment_rounds ASC;
```

10. Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы.
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю.
Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.

```sql
SELECT
MIN(invested_companies) AS min_invested_companies,
MAX(invested_companies) AS max_invested_companies,
AVG(invested_companies) AS avg_invested_companies,
country_code
FROM
fund
WHERE
EXTRACT(YEAR FROM CAST(founded_at AS DATE)) BETWEEN 2010 AND 2012
GROUP BY
country_code
HAVING
MIN(invested_companies) !=0
ORDER BY
AVG(invested_companies) DESC
LIMIT 10;
```

11. Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

```sql
SELECT
p.first_name,
p.last_name,
e.instituition
FROM
people AS p
LEFT JOIN education AS e ON p.id = e.person_id;
```

12. Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

```sql
SELECT c.name,
COUNT(DISTINCT e.instituition) as inst
FROM company AS c
INNER JOIN people AS p ON c.id=p.company_id
INNER JOIN education AS e ON p.id=e.person_id
GROUP BY c.name
ORDER BY inst DESC
LIMIT 5;
```

13. Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

```sql
SELECT DISTINCT name
FROM company
WHERE status LIKE 'closed'
AND id IN (SELECT company_id
FROM funding_round
WHERE is_first_round = 1
AND is_last_round = 1);
```

14. Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

```sql
SSELECT DISTINCT p.id
FROM company AS c
INNER JOIN people AS p ON c.id = p.company_id
WHERE c.status LIKE 'closed'
AND c.id IN (SELECT company_id
FROM funding_round
WHERE is_first_round = 1
AND is_last_round = 1);
```

15. Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

Дополните предыдущий запрос.

```sql
SELECT DISTINCT p.id,
e.instituition
FROM company AS c
INNER JOIN people AS p ON c.id = p.company_id
LEFT JOIN education AS e ON p.id = e.person_id
WHERE c.status LIKE 'closed'
AND c.id IN (SELECT company_id
FROM funding_round
WHERE is_first_round = 1
AND is_last_round = 1)
AND e.instituition IS NOT NULL;
```

16. Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

К предыдущему запросу добавьте группировку и агрегирующую функцию.

```sql
SELECT DISTINCT p.id,
COUNT(e.instituition)
FROM company AS c
INNER JOIN people AS p ON c.id = p.company_id
LEFT JOIN education AS e ON p.id = e.person_id
WHERE c.status LIKE 'closed'
AND c.id IN (SELECT company_id
FROM funding_round
WHERE is_first_round = 1
AND is_last_round = 1)
AND e.instituition IS NOT NULL
GROUP BY p.id;```

17. Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

Используйте подзапрос, чтобы найти среднее от данных, которые вы получили в результате предыдущего запроса.

```sql
SELECT AVG(tab1.count_in)
FROM (SELECT DISTINCT p.id,
COUNT(e.instituition) AS count_in
FROM company AS c
INNER JOIN people AS p ON c.id = p.company_id
LEFT JOIN education AS e ON p.id = e.person_id
WHERE c.status LIKE 'closed'
AND c.id IN (SELECT company_id
FROM funding_round
WHERE is_first_round = 1
AND is_last_round = 1)
AND  e.instituition IS NOT NULL
GROUP BY p.id) AS tab1;
```

18. Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Socialnet.

Этот запрос похож на предыдущий, нужно только изменить условия для среза.

```sql
SELECT AVG(tab1.count_in)
FROM (SELECT DISTINCT p.id,
COUNT(e.instituition) AS count_in
FROM company AS c
INNER JOIN people AS p ON c.id = p.company_id
LEFT JOIN education AS e ON p.id = e.person_id
WHERE c.name LIKE 'Socialnet'
AND  e.instituition IS NOT NULL
GROUP BY p.id
) AS tab1;
```

19. Составьте таблицу из полей:

- `name_of_fund` — название фонда;
- `name_of_company` — название компании;
- `amount` — сумма инвестиций, которую привлекла компания в раунде.

В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

Подсказка

Начните с таблицы `investment` и присоедините к ней таблицы `company` и `fund`. Последней можете присоединить таблицу `funding_round`, но это только рекомендация. Вы можете написать запрос по-своему.

```sql
SELECT f.name AS name_of_fund,
c.name AS name_of_company,
fr.raised_amount AS amount
FROM investment AS i
JOIN company AS c ON i.company_id = c.id
JOIN fund AS f ON i.fund_id = f.id
JOIN funding_round AS fr ON i.funding_round_id = fr.id
WHERE EXTRACT(YEAR FROM fr.funded_at) BETWEEN 2012 AND 2013
AND c.milestones > 6;
```

20. Выгрузите таблицу, в которой будут такие поля:

- название компании-покупателя;
- сумма сделки;
- название компании, которую купили;
- сумма инвестиций, вложенных в купленную компанию;
- доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы.

Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

Самая важная таблица — `acquisition`. Эта таблица содержит информацию о покупках одних компаний другими, поэтому данные из неё пропасть не должны. Выбирайте способ присоединения внимательно.

```sql
SELECT company.name AS acquiring_company,
tab2.price_amount,
tab2.acquired_company,
tab2.funding_total,
ROUND(tab2.price_amount / tab2.funding_total)
FROM
(
SELECT c.name AS acquired_company,
c.funding_total,
tab1.acquiring_company_id,
tab1.price_amount
FROM company AS c
RIGHT JOIN (
SELECT acquiring_company_id,
acquired_company_id,
price_amount
FROM acquisition
WHERE price_amount > 0
) AS tab1 ON c.id = tab1.acquired_company_id
) AS tab2 LEFT JOIN company ON company.id  = tab2.acquiring_company_id
WHERE tab2.funding_total > 0
ORDER BY  tab2.price_amount DESC, tab2.acquired_company
LIMIT 10;
```

21. Выгрузите таблицу, в которую войдут названия компаний из категории `social`, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.


Отберите компании, которые участвовали в раундах с 2010 по 2013 год включительно и привлекли любую сумму инвестиций, не равную нулю. Вам понадобятся таблицы `company` и `funding_round`.

```sql
SELECT c.name,
tab1.month
FROM company AS c
RIGHT JOIN (
SELECT company_id,
EXTRACT(MONTH FROM funded_at) AS month
FROM funding_round
WHERE EXTRACT(YEAR FROM funded_at) BETWEEN 2010 AND 2013
AND raised_amount != 0) AS tab1 ON c.id = tab1.company_id
WHERE c.category_code LIKE 'social';
```

22. Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:

- номер месяца, в котором проходили раунды;
- количество уникальных названий фондов из США, которые инвестировали в этом месяце;
- количество компаний, купленных за этот месяц;
- общая сумма сделок по покупкам в этом месяце.


Чтобы вычислить номер месяца проведения раундов, используйте значения в поле `funded_at` в таблице `funding_round`. Два последних поля — с количеством купленных компаний и с суммой сделок — можно получить, взяв данные из таблицы `acquisition`. Используйте общие табличные выражения — будет легче не запутаться в присоединении.
```sql
WITH
-- выбираем месяц инвестиционных раундов в 2010-2013 гг
tab1 AS (SELECT EXTRACT(MONTH FROM funded_at) AS month,
id AS funding_round_id
FROM funding_round
WHERE EXTRACT(YEAR FROM funded_at) BETWEEN 2010 AND 2013
),

- - считаем кол-во купленных и общую сумму по сделкам за 2010-2013 гг в разрезе месяца
tab2 AS (SELECT EXTRACT(MONTH FROM acquired_at) AS month,
COUNT(acquired_company_id) AS count_acquired,
SUM(price_amount) AS total_amount
FROM acquisition
WHERE EXTRACT(YEAR FROM acquired_at) BETWEEN 2010 AND 2013
GROUP BY EXTRACT(MONTH FROM acquired_at)
),
- - ищем фонды из США
tab3 AS (SELECT i.funding_round_id,
[f.name](http://f.name/)
FROM investment AS i
JOIN fund AS f ON [f.id](http://f.id/) = i.fund_id
WHERE fund_id IN (SELECT id
FROM fund
WHERE country_code LIKE 'USA')
),

tab4 AS (SELECT month,
COUNT(DISTINCT name) AS count_USA
FROM tab1
LEFT JOIN tab3 ON tab1.funding_round_id = tab3.funding_round_id
GROUP BY month)

SELECT tab4.month,
tab4.count_USA,
tab2.count_acquired,
tab2.total_amount
FROM tab4
LEFT JOIN tab2 ON tab4.month = tab2.month;
```

23. Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

```sql
WITH
inv_2011 AS (SELECT AVG(funding_total) AS inv_2011,
country_code
FROM company
GROUP BY country_code,
EXTRACT(YEAR FROM founded_at)
HAVING EXTRACT(YEAR FROM founded_at) = 2011),

inv_2012 AS (SELECT AVG(funding_total) AS  inv_2012,
                  country_code
           FROM company
           GROUP BY country_code,
           EXTRACT(YEAR FROM founded_at)
           HAVING EXTRACT(YEAR FROM founded_at) = 2012),

  inv_2013 AS (SELECT AVG(funding_total) AS inv_2013,
                  country_code
           FROM company
           GROUP BY country_code,
           EXTRACT(YEAR FROM founded_at)
           HAVING EXTRACT(YEAR FROM founded_at) = 2013)
SELECT inv_2011.country_code,
inv_2011.inv_2011,
inv_2012.inv_2012,
inv_2013.inv_2013

FROM inv_2011
INNER JOIN inv_2012 ON inv_2011.country_code = inv_2012.country_code
INNER JOIN inv_2013 ON inv_2011.country_code = inv_2013.country_code
ORDER BY inv_2011.inv_2011 DESC;
```